# ROS TF study

ROSを用いてTFを使いこなす練習台。

## 参考になる文献

公式度が高いものほど上に記述する。なるべく日本語文献をチョイスする。

- [TF tutorial for Pyton](http://wiki.ros.org/tf/TfUsingPython)
- [Python版TFパッケージのドキュメント（古い可能性有り）](http://docs.ros.org/jade/api/tf/html/python/tf_python.html)
- [TFによる座標変換 解説と演習](https://industrial-training-jp.readthedocs.io/ja/latest/_source/session3_JP/Coordinate-Transforms-using-TF_JP.html)
- [公式・非公式の壁]()
- [Slideshare TF完全解説](https://www.slideshare.net/kojiterada5/tftf2)

どれを読んでも良いが手っ取り早く全体像を把握したいなら，[Slideshare TF完全解説](https://www.slideshare.net/kojiterada5/tftf2)がわかりやすいと思う。


In [2]:
# 使いそうなパッケージ
import rospy
import geometry_msgs.msg
import tf
import tf2_ros

# 1. TF Transformer Training

- 補間や変形に使う強力なツール，`tf.Transformer`を試す。
  - 一部  http://docs.ros.org/jade/api/tf/html/python/tf_python.html を参照。


```python
class tf.Transformer(interpolating, cache_time = rospy.Duration(10))
```

    - interpolatingは補完をするかどうか。基本True。
    - chache_timeは過去のTransformをどれだけ長く保持するか。

In [3]:
# TFを作成
t = tf.Transformer(True, rospy.Duration(10.0))
t.getFrameStrings()

[]

In [4]:
# 空のTransformを作成
m = geometry_msgs.msg.TransformStamped()
print(m)

header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
child_frame_id: ''
transform: 
  translation: 
    x: 0.0
    y: 0.0
    z: 0.0
  rotation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 0.0


In [5]:
# Transformに値を詰めていく
m.header.frame_id = 'THISFRAME'
m.child_frame_id = 'CHILD'
m.transform.translation.x = 2.71828183
m.transform.rotation.w = 1.0 # これを入れないと無効なクォータニオンになってしまう。
print(m)

# TfにTransformを詰める
t.setTransform(m)
t.getFrameStrings() # ちょっと結果違う？

header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: "THISFRAME"
child_frame_id: "CHILD"
transform: 
  translation: 
    x: 2.71828183
    y: 0.0
    z: 0.0
  rotation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0


['CHILD']

## 1-1 `lookupTransform`：位置の参照と補間

- 時間と位置を指定して変形を取得 `lookupTransform`
- 未来の時間を指定するとエラーが出る
- 複数のTransformをSetすると補間ができる。

In [6]:
# 時間0（現在）における位置。
t.lookupTransform('THISFRAME', 'CHILD', rospy.Time(0))

([2.71828183, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0])

In [7]:
# 時間0（現在）における位置。ただし反転版
t.lookupTransform( 'CHILD','THISFRAME', rospy.Time(0))

([-2.71828183, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0])

In [16]:
# 時間1（未来時間）における位置。→ この時点では最新ではないのでエラーが出る。
t.lookupTransform('THISFRAME', 'CHILD', rospy.Time(1))

ExtrapolationException: Lookup would require extrapolation into the future.  Requested time 1.000000000 but the latest data is at time 0.000000000, when looking up transform from frame [CHILD] to frame [THISFRAME]

In [8]:
# 時間4sにおけるTransform
m2 = geometry_msgs.msg.TransformStamped()
m2.header.frame_id = 'THISFRAME'
m2.child_frame_id = 'CHILD'
m2.transform.translation.y = 1
m2.transform.rotation.w = 1.0 # これを入れないと無効なクォータニオンになってしまう。
# Timestampのセット
m2.header.stamp.set(secs=4,nsecs=0)
print(m2)

header: 
  seq: 0
  stamp: 
    secs: 4
    nsecs:         0
  frame_id: "THISFRAME"
child_frame_id: "CHILD"
transform: 
  translation: 
    x: 0.0
    y: 1
    z: 0.0
  rotation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0


In [9]:
# TfにTransformを詰める
t.setTransform(m2)
# 'THISFRAME', 'CHILD'間の最新の時間を確認
t.getLatestCommonTime('THISFRAME', 'CHILD')

rospy.Time[4000000000]

In [10]:
# 時間1（未来時間）における位置 → うまいこと補間されている。
t.lookupTransform('THISFRAME', 'CHILD', rospy.Time(1))

([2.0387113725, 0.25, 0.0], [0.0, 0.0, 0.0, 1.0])

In [11]:
# 時間の比較
t.getLatestCommonTime('THISFRAME', 'CHILD') > rospy.Time(1.0)

True

### 1-1 まとめ

- tをtfのObjectとする。
- `t.lookupTransform('PARENTFRAME', 'CHILD', rospy.Time(Time))`で適当な時間でのTransformの補間ができる。
-  `t.getLatestCommonTime('THISFRAME', 'CHILD') > rospy.Time(知りたい時間)`がTrueなら取得できうる。

## 1-2 TransformListener：Publishされている/tf情報を収集する。

- 現実にはtfは手動で入れることはなくて，Publishされているtfを元に先程の操作を行うことが多い。
- こいつのシミュレーションはちょっと面倒なのでSkip

In [13]:
# init node needed
rospy.init_node("test", anonymous=True)
tl=tf.TransformListener() 

In [14]:
# 有る時間の値が欲しい → 値が出るまで待機，実行
# ちなみにこれは init_node()していないと動作しないので実行しないで
now = rospy.Time.now()
print(now)

# tf が利用可能になるまで 1秒待つ。架空のframeなので実行不可。
tl.waitForTransform("/frame1","/frame2",rospy.Duration(1.0))

1600145395592798948


In [36]:
# Show all frames that this node can access
tl.allFramesAsDot()

'digraph G {\n"cam_d435i_1_link" -> "cam_d435i_1_depth_frame"[label="Broadcaster: /play_1600147293913242491\\nAverage rate: 10000.000 Hz\\nMost recent transform: 0.000 \\nBuffer length: 0.000 sec\\n"];\n"base_link" -> "cam_d435i_1_link"[label="Broadcaster: /play_1600147293913242491\\nAverage rate: 10000.000 Hz\\nMost recent transform: 0.000 \\nBuffer length: 0.000 sec\\n"];\n"cam_d435i_1_depth_frame" -> "cam_d435i_1_depth_optical_frame"[label="Broadcaster: /play_1600147293913242491\\nAverage rate: 10000.000 Hz\\nMost recent transform: 0.000 \\nBuffer length: 0.000 sec\\n"];\n"cam_d435i_1_link" -> "cam_d435i_1_color_frame"[label="Broadcaster: /play_1600147293913242491\\nAverage rate: 10000.000 Hz\\nMost recent transform: 0.000 \\nBuffer length: 0.000 sec\\n"];\n"cam_d435i_1_aligned_depth_to_color_frame" -> "cam_d435i_1_color_optical_frame"[label="Broadcaster: /play_1600147293913242491\\nAverage rate: 10000.000 Hz\\nMost recent transform: 0.000 \\nBuffer length: 0.000 sec\\n"];\n"cam_d43

## TFのBroadcast

- これらの利点を享受するためにはPoseではなくTfとしてBroadcastするのが良い。


In [21]:
#Poseの作成
p = geometry_msgs.msg.PoseStamped()
p.header.frame_id = "RobotPoseTest"
p.pose.position.x = 0.90
p.pose.position.y = 0.30
p.pose.position.z = 1.2
p.pose.orientation.x=0.0
p.pose.orientation.y=0.0
p.pose.orientation.z=0.0
p.pose.orientation.w=1.0
p

header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: "RobotPoseTest"
pose: 
  position: 
    x: 0.9
    y: 0.3
    z: 1.2
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0

- memo: StampとRos::Timeは同じ型

In [32]:
# PoseのTransformへの変換
child_frame = "TestChildLink"
parent_frame = p.header.frame_id

# Pose Broad caster
br = tf.TransformBroadcaster()
br.sendTransform(
    (p.pose.position.x, p.pose.position.y, p.pose.position.z),
    (p.pose.orientation.x, p.pose.orientation.y, p.pose.orientation.z, p.pose.orientation.w),
    p.header.stamp,  
    child_frame,    # maybe base_link
    parent_frame    # equivalent of odom
    )

'digraph G {\n"cam_d435i_1_link" -> "cam_d435i_1_depth_frame"[label="Broadcaster: /play_1600147293913242491\\nAverage rate: 10000.000 Hz\\nMost recent transform: 0.000 \\nBuffer length: 0.000 sec\\n"];\n"base_link" -> "cam_d435i_1_link"[label="Broadcaster: /play_1600147293913242491\\nAverage rate: 10000.000 Hz\\nMost recent transform: 0.000 \\nBuffer length: 0.000 sec\\n"];\n"cam_d435i_1_depth_frame" -> "cam_d435i_1_depth_optical_frame"[label="Broadcaster: /play_1600147293913242491\\nAverage rate: 10000.000 Hz\\nMost recent transform: 0.000 \\nBuffer length: 0.000 sec\\n"];\n"cam_d435i_1_link" -> "cam_d435i_1_color_frame"[label="Broadcaster: /play_1600147293913242491\\nAverage rate: 10000.000 Hz\\nMost recent transform: 0.000 \\nBuffer length: 0.000 sec\\n"];\n"cam_d435i_1_aligned_depth_to_color_frame" -> "cam_d435i_1_color_optical_frame"[label="Broadcaster: /play_1600147293913242491\\nAverage rate: 10000.000 Hz\\nMost recent transform: 0.000 \\nBuffer length: 0.000 sec\\n"];\n"cam_d43

# 2 tf:Transformations
- tf.transformations 以下に存在する諸々のパッケージ [Wiki](http://wiki.ros.org/tf/Overview/Transformations)
- Tfをもちいた同次変換（Homogeneous Transformation）
- Tfを用いた座標変換

### TF同士の足し引き？

同次変換行列に変換後，行列の掛け算にて実施できる。

In [15]:
# m+m2 みたいな計算はできない。


Exception AttributeError: "TransformListener instance has no attribute 'tf_sub'" in <bound method TransformListener.__del__ of <tf2_ros.transform_listener.TransformListener instance at 0x7f777ca76370>> ignored


TypeError: unsupported operand type(s) for +: 'TransformStamped' and 'TransformStamped'

In [16]:
tf.transformations.

AttributeError: 'module' object has no attribute 'poseMsg2ToTF'

## Poseと戯れる

In [19]:
#This is object2
p = geometry_msgs.msg.PoseStamped()
p.header.frame_id = "Robot"
p.pose.position.x = 0.90
p.pose.position.y = 0.30
p.pose.position.z = 1.2
p.pose.orientation.x=0.0
p.pose.orientation.y=0.0
p.pose.orientation.z=0.0
p.pose.orientation.w=1.0
p

header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: "Robot"
pose: 
  position: 
    x: 0.9
    y: 0.3
    z: 1.2
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0